# Projectile
## Second exercise

## Introduction to optimization and operations research.

Michel Bierlaire


A projectile is launched vertically at a rate of 50 meters per second in the absence of wind. After how long will it
reach again the ground, that is, its **lowest** altitude?

Import necessary packages.

In [ ]:
from scipy.optimize import minimize, OptimizeResult

from teaching_optimization.plot_function import plot_function



First implement the calculation of the altitude of the projectile using the formula for uniformly accelerated
movement:  $$f(t) = x_o + v_0 t -\frac{g}{2} t^2,$$
where $x_0$ is the initial altitude, $v_0$ the initial speed, and $g$ the acceleration due
to gravity.

Define a constant.

In [ ]:
GRAVITY = 9.81



Write a function to calculate the height of the projectile, using the formula.

In [ ]:
def height(time: float, initial_altitude: float, initial_speed: float) -> float:
    """
    Calculate the height of the projectile, using the formula.

    :param time: time at which we need the height.
    :param initial_altitude: initial altitude x_0.
    :param initial_speed: initial speed v_0.
    :return: height.
    """
    return (
        initial_altitude + initial_speed * time - GRAVITY * time * time / 2
    )



Define the objective function. The "decision" variable is the time.

In [ ]:
def objective_function(x: float) -> float:
    """Objective function of the optimization problem.
    Here, the "decision" variable is the time. Here, we do not change the sign of
    the objective function as we need to solve a minimization problem.
    """
    return height(time=x, initial_altitude=0, initial_speed=50)



Plot the objective function. Make a visual guess of the solution.

In [ ]:
plot_function(
    my_function=objective_function,
    label='Altitude',
    x_label='Time (sec.)',
    y_label='Altitude (m.)',
    x_min=0,
    x_max=12,
    y_min=0,
    y_max=150,
)


According to the plot, the solution we are looking for should be something slightly larger than 10 sec.

Initial solution

In [ ]:
x0 = 0.0


Run the algorithm.

In [ ]:
the_result = minimize(fun=objective_function, x0=x0)



Function that prints the results.

In [ ]:
def print_results(optimization_result: OptimizeResult) -> None:
    # Print the raw results.
    print(optimization_result)
    print()
    # Print the solution with 3 significant digits.
    print(f'Elapsed time:     {optimization_result.x[0]:.3g} sec.')
    print(f'Altitude reached: {optimization_result.fun:.3g} m.')



Expected results:

- Elapsed time:     10.2 sec.
- Altitude reached: 0 m.

In [ ]:
print_results(optimization_result=the_result)


Those results do not make sense.
Try to fix the problem by:

- defining constraints,
- changing the starting point

Actually, the above optimization problem is **unbounded** and no solution exists. Our formulation is incomplete.
Time can only go forward, and therefore must be non-negative.
And altitude cannot be negative either. This has to be mentioned to the algorithm.

First, we impose a lower bound on the variable.

In [ ]:
bounds = [(0, None)]





Second, we introduce a constraint on the altitude. In ``scipy``, inequality constraints are coded
as :math:`c(point) \geq 0`. In our case, :math:`c(point)` represents the altitude.

In [ ]:
def inequality_constraint(x: float) -> float:
    """Altitude of the projectile"""
    return height(time=x, initial_altitude=0, initial_speed=50)





Run the algorithm.

In [ ]:
x0 = 0
the_result = minimize(
    fun=objective_function,
    x0=x0,
    constraints={'type': 'ineq', 'fun': inequality_constraint},
    bounds=bounds,
)


Note that 'success' is set to True.

In [ ]:
print_results(optimization_result=the_result)


The algorithm has returned a solution which is now correct. But this is not the one that we needed.
We know from the previous exercise that the projectile reaches its highest altitude after 5.1 sec. Therefore, we can
impose, without loss of generality, that the time should be at least 1 sec.

In [ ]:
x0 = 0
bounds = [(1, None)]
the_result = minimize(
    fun=objective_function,
    x0=x0,
    constraints={'type': 'ineq', 'fun': inequality_constraint},
    bounds=bounds,
)


Note that 'success' is set to True.

In [ ]:
print_results(optimization_result=the_result)


This is again incorrect. According to the plot, the solution should be something larger than 10 sec.
The problem is that the algorithm has been blocked in a local optimum. In order to escape from it, we change the
starting point,

In [ ]:
x0 = 6.0
bounds = [(1, None)]
the_result = minimize(
    fun=objective_function,
    x0=x0,
    constraints={'type': 'ineq', 'fun': inequality_constraint},
    bounds=bounds,
)


Note that 'success' is set to True.

In [ ]:
print_results(optimization_result=the_result)


This is the expected result. Note that the altitude is not exactly zero, but a number very close to zero.
Such "numerical features" are happening often in optimization.